# Single CPP File Build

In this example we will show how we can wrap a simple CPP project with the default parameters, which will enable us to just provide a single CPP file.

If you want to wrap an existing library, or you have more complex requirements please refer to the ["Build system override CPP Wrapper example"]().

You can read about how to configure your environment in the [CPP Wrapper page]().

## CPP Wrapper

The only thing that we will need for this, is a single CPP file. 

To ensure we align with the defaults, the name of our CPP file has to be `SeldonPackage.cpp`.

The contents of our file are quire minimal - namely:

In [3]:
%%writefile SeldonPackage.cpp
#include "seldon/SeldonModel.hpp"

class ModelClass : public seldon::SeldonModelBase {

    seldon::protos::SeldonMessage predict(seldon::protos::SeldonMessage &data) override {
        return data;
    }
};

SELDON_DEFAULT_PYBIND_MODULE(ModelClass)


Overwriting SeldonPackage.cpp


In this file we basically have to note the following key points:

* We import `"seldon/SeldonModel.hpp"` which is from the Seldon package
* For the defaults the name of our class has to be `"ModelClass"`
* We extend the `SeldonModelBase` class which processes the protos for us
* We override the `predict()` function which provides the raw protos
* We register our class as `SELDON_DEFAULT_PYBIND_MODULE`

## Build Seldon Microservice

We can now build our seldon microservice using `s2i`:

In [6]:
!s2i build . seldonio/s2i-cpp-build:0.0.1 seldonio/simple-cpp:0.1

---> Installing application source...
---> Installing application ...
Looking in links: /whl
Obtaining file:///microservice
Running setup.py develop for SeldonPackage
Created wheel for PTable: filename=PTable-0.9.2-cp37-none-any.whl size=22906 sha256=f8ceb5d135fae9aad1de576924d11560c8a004a59c489b980412a974eeffd694
Stored in directory: /root/.cache/pip/wheels/22/cc/2e/55980bfe86393df3e9896146a01f6802978d09d7ebcba5ea56
Successfully built PTable
created path: ./licenses/license_info.csv
created path: ./licenses/license.txt
Build completed successfully


## Test our model locally by running docker

In [7]:
!docker run --name "simple_cpp" -d --rm -p 5000:5000 cppmodel:0.1

19f989a0ae389e041c4daf915067016a6ab8dc569873948a1099442a249ec502


### Send request (which should return the same value)

In [12]:
!curl -X POST -H 'Content-Type: application/json' \
   -d '{"strData":"hello"}' \
    http://localhost:5000/api/v1.0/predictions

{"strData":"hello"}

### Clean up

In [13]:
!docker rm -f "simple_cpp"

simple_cpp


## Deploy to seldon

In [15]:
%%bash
kubectl apply -f - << END
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: simple-cpp
spec:
  predictors:
  - componentSpecs:
    - spec:
        containers:
          - image: seldonio/simple-cpp:0.1
            name: classifier
    engineResources: {}
    graph:
      name: classifier
      type: MODEL
    name: default
    replicas: 1
END

seldondeployment.machinelearning.seldon.io/simple-cpp created


In [16]:
!curl -X POST -H 'Content-Type: application/json' \
   -d '{"strData":"hello"}' \
    http://localhost:80/seldon/default/simple-cpp/api/v1.0/predictions

{"strData":"hello"}